In [15]:
import pandas as pd

Reading in training data


In [16]:
df1 = pd.read_csv('finnum/train.csv')
x = df1.index
df2 = pd.read_csv('finnum/dev.csv')
df = pd.concat([df1,df2], ignore_index = True)
df.head(10)

,id,idx,tweet,category,subcategory,target_num
0,98221616,4976,$ARNA APD334 for Amyotrophic Lateral Sclerosis...,Product Number,Product Number,334
1,82321187,9839,"$OCLR Noob investor that i am, put a 7.38 stop...",Monetary,stop loss,7.38
2,103328840,1455,$ES_F $SPY Bias-2 bearish and the DLT-1 DRR ar...,Product Number,Product Number,1
3,104840294,1111,$TMUS its acquisition of Layer3 TV The purchas...,Product Number,Product Number,5
4,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Percentage,relative,14
5,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Quantity,Quantity,4
6,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Monetary,forecast,5
7,94249158,1372,$SEED L2 Capital deal is real savvy. It takes ...,Temporal,date,33
8,100979260,505,$BTE $BTE.CA $MEG.CA $CPG $CPG.CA $CJ.CA - 4th...,Temporal,date,4
9,100775772,1210,$WRN My fav $WRN pattern on my watchlist for 1...,Temporal,date,11


Make new column for encoding categories as numbers

In [17]:
df['cat_num'] = df['category'].astype('category').cat.codes

Preprocessing tweets by lowercasing

In [18]:
df['lower'] = [x.lower() for x in df.tweet]

In [19]:
df['lower'].head()

0    $arna apd334 for amyotrophic lateral sclerosis...
1    $oclr noob investor that i am, put a 7.38 stop...
2    $es_f $spy bias-2 bearish and the dlt-1 drr ar...
3    $tmus its acquisition of layer3 tv the purchas...
4    $twtr ^buy  $wstl 68c up 14%  4 time avg vol. ...
Name: lower, dtype: object

Replacing target with <num\>

In [20]:
import re
def flagNum(x):
    text = x.lower
    outNum = str(x.target_num)
    text_out = re.sub(r'(?<=\D)'+outNum+'(?=\D)', ' <num> ', text)
    #text_out = text.replace('\D('+outNum+')\D', ' <num> ')
    return text_out

Applying to training, making this into a new column

In [21]:
df['mod'] = df.apply(lambda x: flagNum(x), axis = 1)

In [22]:
df['mod'].head()

0    $arna apd <num>  for amyotrophic lateral scler...
1    $oclr noob investor that i am, put a  <num>  s...
2    $es_f $spy bias-2 bearish and the dlt- <num>  ...
3    $tmus its acquisition of layer3 tv the purchas...
4    $twtr ^buy  $wstl 68c up  <num> %  4 time avg ...
Name: mod, dtype: object

Filtering out numbers and words less than 3 characters long

In [23]:
import string
def textPuncandNum(text):
    table = str.maketrans('', '', string.punctuation)
    text = re.sub(r'[0-9]+', '', text)
    text = text.split()
    text = [word for word in text if len(word.translate(table))>2]
    return ' '.join(text)
stripped = [textPuncandNum(text) for text in df['mod']]

In [24]:
split = [x.split() for x in stripped]

In [25]:
def replace_contraction(text):
    contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'can not'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
                         (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not') ]
    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    return text
def replace_links(text, filler=' '):
        text = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*',
                      filler, text).strip()
        return text
def remove_numbers(text):
    text = ''.join([i for i in text if not i.isdigit()])
    return text

In [26]:
import re
import numpy as np
def cleanText(text):
    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = replace_contraction(text)
    text = replace_links(text, "link")
    text = remove_numbers(text)
    text = re.sub(r'[,!%^&*)(|";:.?\'\\}{]',"",text)
    text = text.lower()
    return text

In [27]:
words = df["mod"].apply(cleanText)
max_length = max(len(lst.split()) for lst in words)
tokens_length = np.asarray([len(x.split()) for x in words])
t_pad = np.asarray([lst.split() + [''] * (max_length - len(lst.split())) for lst in words])

## Implementing Convolutional Layer

In [28]:
import tensorflow as tf
import tensorflow_hub as hub
batch_size = 64

W0415 14:58:06.689770 10372 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [29]:
tf.reset_default_graph()

In [30]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
def ELMoEmbedding(x,y):
    return elmo(inputs = {"tokens": tf.squeeze(tf.cast(x, tf.string)), "sequence_len":tf.cast(y, tf.int32)}, signature="tokens", as_dict=True)["elmo"]

Instructions for updating:
Colocations handled automatically by placer.


W0415 14:58:13.793577 10372 deprecation.py:323] From C:\Users\justi\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Creating the embedding matrix for any input


In [31]:
trainInp = t_pad[x, :]
valInp = t_pad[len(x):, :]

t_len = tokens_length[x]
v_len = tokens_length[len(x):]

labels = pd.get_dummies(df['cat_num'][x])
labels2 = pd.get_dummies(df['cat_num'][len(x):])

Batching and creating iterators

In [32]:
train = (trainInp, t_len, labels)
val = (valInp, v_len, labels2)

# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

# create validation Dataset and batch it
val_data = tf.data.Dataset.from_tensor_slices(val)
val_data = val_data.shuffle(10000) # if you want to shuffle your data
val_data = val_data.batch(batch_size)

# create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
txt, length, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
val_init = iterator.make_initializer(val_data)

In [33]:
elmos = ELMoEmbedding(txt, length)
embedded_chars_expanded = tf.expand_dims(elmos, -1)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0415 14:58:20.930797 10372 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [34]:
embedded_chars_expanded

<tf.Tensor 'ExpandDims:0' shape=(?, ?, 1024, 1) dtype=float32>

In [35]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(train_init)
    x = elmos.eval()

Implementing looped convolution

In [36]:
pooled_outputs = []
filter_sizes = [2,3,4,5]
embedding_size = 1024
num_filters = 64
max_length = 32
for filter_size in filter_sizes:
    filter_shape = [filter_size, embedding_size, 1, num_filters]
    W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name='W')
    b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name = 'b')
    conv = tf.nn.conv2d(
        embedded_chars_expanded,
        tf.cast(W,tf.float32),
        strides=[1, 1, 1, 1],
        padding='VALID',
        name='conv')
    relu = tf.nn.relu(tf.nn.bias_add(conv, tf.cast(b,tf.float32)), name="relu")
    pooled = tf.nn.max_pool(
        relu,
        ksize=[1, max_length - filter_size + 1, 1, 1],
        strides=[1, 1, 1, 1],
        padding='VALID',
        name="pool")
    pooled_outputs.append(pooled)

In [37]:
pooled_outputs

[<tf.Tensor 'pool:0' shape=(?, ?, 1, 64) dtype=float32>,
 <tf.Tensor 'pool_1:0' shape=(?, ?, 1, 64) dtype=float32>,
 <tf.Tensor 'pool_2:0' shape=(?, ?, 1, 64) dtype=float32>,
 <tf.Tensor 'pool_3:0' shape=(?, ?, 1, 64) dtype=float32>]

Combining separate convolutional layers into 1 feed forward input

In [38]:
num_filters_total = num_filters * len(filter_sizes)
combined = tf.concat(pooled_outputs, 3)
combined_flat = tf.reshape(combined, [-1, num_filters_total])

In [39]:
combined_flat

<tf.Tensor 'Reshape:0' shape=(?, 256) dtype=float32>

Adding dense layers

In [40]:
conn = tf.layers.dense(combined_flat, 100, activation = 'relu')
conn2 = tf.layers.dense(conn, len(set(df.cat_num)))

Instructions for updating:
Use keras.layers.dense instead.


W0415 14:58:35.298825 10372 deprecation.py:323] From <ipython-input-40-4512dc98bc35>:1: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


In [41]:
conn

<tf.Tensor 'dense/Relu:0' shape=(?, 100) dtype=float32>

Implementing cross entropy, loss, and optimization

In [42]:
n_epochs = 20
entropy = tf.nn.softmax_cross_entropy_with_logits(labels = label, logits = conn2)
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



W0415 14:58:35.394562 10372 deprecation.py:323] From <ipython-input-42-8cbb1315fd70>:2: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Prediction setup

In [43]:
preds = tf.nn.softmax(conn2)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))

Running Neural Net

In [45]:
with tf.Session() as sess:
    #start_time = time.time()
    sess.run(tf.global_variables_initializer())
    # train the model n_epochs times

    for i in range(n_epochs): 
        
        sess.run(train_init)# drawing samples from train_data
        total_loss = 0
        total_right = 0
        n_batches = 0
        totalright = 0
        totalvalright = 0
        nvalbatches = 0
        totalvalloss = 0
        try:
            while True:
                #summary,acc,_, l = sess.run([summary_op,accuracy,optimizer, loss]) #use with scalar summary
                acc,_, l = sess.run([accuracy, optimizer, loss])                
                total_loss += l
                total_right += acc
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        
        sess.run(val_init)
        try:
            while True:
                accval,valloss = sess.run([accuracy, loss])
                totalvalright += accval
                nvalbatches += 1
                totalvalloss += valloss
        except tf.errors.OutOfRangeError:
            pass

        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        print('Accuracy {0}: {1}'.format(i, total_right/n_batches))    
        print('Average val loss epoch {0}: {1}'.format(i, totalvalloss/nvalbatches))
        print('Val_Accuracy {0}: {1}'.format(i, totalvalright/nvalbatches)) 
    prediction = sess.run(preds, feed_dict={txt: valInp})
    prediction = np.asarray(prediction)
   


Average loss epoch 0: 2.2691607997530983
Accuracy 0: 0.365880952278773
Average val loss epoch 0: 1.546849896510442
Val_Accuracy 0: 0.41640624900658924
Average loss epoch 1: 1.2935858357520331
Accuracy 1: 0.5229761907032557
Average val loss epoch 1: 1.3394907017548878
Val_Accuracy 1: 0.5057291661699613
Average loss epoch 2: 1.0413385851042611
Accuracy 2: 0.6235476192973909
Average val loss epoch 2: 1.2259408334891002
Val_Accuracy 2: 0.5494791666666666
Average loss epoch 3: 0.847709482056754
Accuracy 3: 0.7075178572109767
Average val loss epoch 3: 1.159574717283249
Val_Accuracy 3: 0.5791666656732559
Average loss epoch 4: 0.7006366079761869
Accuracy 4: 0.7736488092513312
Average val loss epoch 4: 1.082480639219284
Val_Accuracy 4: 0.5963541666666666
Average loss epoch 5: 0.5776882299355098
Accuracy 5: 0.821434523945763
Average val loss epoch 5: 1.031128153204918
Val_Accuracy 5: 0.6226562509934107
Average loss epoch 6: 0.4765319370088123
Accuracy 6: 0.8659285715648106
Average val loss epoch

KeyboardInterrupt: 

Calculating accuracy for train

In [43]:
np.mean(np.equal(np.argmax(prediction, 1), labels2.idxmax(axis = 1)))

0.5873655913978495